In [4]:
import numpy as np
import pandas as pd

In [5]:
aisles_df = pd.read_csv('../data/original/aisles.csv')
departments_df = pd.read_csv('../data/original/departments.csv')

order_products_prior_df = pd.read_csv('../data/original/order_products__prior.csv')
order_products_train_df = pd.read_csv('../data/original/order_products__train.csv')

orders_df = pd.read_csv('../data/original/orders.csv')
products_df = pd.read_csv('../data/original/products.csv')

In [6]:
order_count_df = orders_df[orders_df['eval_set'] == 'prior'].groupby('user_id').count()

In [7]:
user_feature_df = pd.DataFrame(index=order_count_df.index)

In [8]:
user_feature_df['order_count'] = order_count_df['order_id']

In [9]:
base_feature_df = pd.merge(order_products_prior_df, orders_df, how='left', on="order_id")

In [10]:
base_feature_df = pd.merge(base_feature_df, products_df, how='left', on='product_id')

In [11]:
base_feature_df = pd.merge(base_feature_df, products_df, how='left', on='product_id')

In [12]:
base_feature_df

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name_x,aisle_id_x,department_id_x,product_name_y,aisle_id_y,department_id_y
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,Natural Sweetener,17,13
5,2,17794,6,1,202279,prior,3,5,9,8.0,Carrots,83,4,Carrots,83,4
6,2,40141,7,1,202279,prior,3,5,9,8.0,Original Unflavored Gelatine Mix,105,13,Original Unflavored Gelatine Mix,105,13
7,2,1819,8,1,202279,prior,3,5,9,8.0,All Natural No Stir Creamy Almond Butter,88,13,All Natural No Stir Creamy Almond Butter,88,13
8,2,43668,9,0,202279,prior,3,5,9,8.0,Classic Blend Cole Slaw,123,4,Classic Blend Cole Slaw,123,4
9,3,33754,1,1,205970,prior,16,5,17,12.0,Total 2% with Strawberry Lowfat Greek Strained...,120,16,Total 2% with Strawberry Lowfat Greek Strained...,120,16


In [13]:
product_feature_df = base_feature_df.groupby(['user_id', 'product_id'])['order_id'].count().reset_index()

In [14]:
product_feature_df = product_feature_df.rename(columns={'order_id':'product_count'})

In [15]:
product_feature_df['product_buy_rate'] = (base_feature_df.groupby(['user_id', 'product_id'])['order_id'].count() / user_feature_df['order_count']).values

In [16]:
product_feature_df = pd.merge(product_feature_df, products_df, how='left', on='product_id')

In [17]:
product_feature_df['add_cart_order_mean'] = (base_feature_df.groupby(['user_id', 'product_id'])['add_to_cart_order']).mean().values
product_feature_df['avg_add_cart_order_std'] = (base_feature_df.groupby(['user_id', 'product_id'])['add_to_cart_order']).std().values

In [18]:
product_feature_df['avg_add_cart_order_std'].fillna(0,inplace=True)

In [19]:
product_count_per_order_df = pd.DataFrame(base_feature_df.groupby(['user_id', 'order_id']).count().groupby('user_id')['product_id'].mean()).reset_index()

In [20]:
product_count_per_order_df.rename(columns={'product_id':'product_count_per_order_mean'}, inplace=True)

In [21]:
product_feature_df = pd.merge(product_feature_df, product_count_per_order_df, how='left', on='user_id')

In [22]:
product_feature_df

,user_id,product_id,product_count,product_buy_rate,product_name,aisle_id,department_id,add_cart_order_mean,avg_add_cart_order_std,product_count_per_order_mean
0,1,196,10,1.000000,Soda,77,7,1.400000,0.966092,5.900000
1,1,10258,9,0.900000,Pistachios,117,19,3.333333,1.322876,5.900000
2,1,10326,1,0.100000,Organic Fuji Apples,24,4,5.000000,0.000000,5.900000
3,1,12427,10,1.000000,Original Beef Jerky,23,19,3.300000,2.406011,5.900000
4,1,13032,3,0.300000,Cinnamon Toast Crunch,121,14,6.333333,1.527525,5.900000
5,1,13176,2,0.200000,Bag of Organic Bananas,24,4,6.000000,2.828427,5.900000
6,1,14084,1,0.100000,Organic Unsweetened Vanilla Almond Milk,91,16,2.000000,0.000000,5.900000
7,1,17122,1,0.100000,Honeycrisp Apples,24,4,6.000000,0.000000,5.900000
8,1,25133,8,0.800000,Organic String Cheese,21,16,4.000000,0.534522,5.900000
9,1,26088,2,0.200000,Aged White Cheddar Popcorn,23,19,4.500000,0.707107,5.900000


In [34]:
def calculateBoughtNumberOfTimePeriodMean(order_number_series):
    if order_number_series.count() <= 1:
        return 100
    else:
        rate = (order_number_series.max() - order_number_series.min()) / (order_number_series.count() - 1)
        return rate

In [44]:
def calculateBoughtNumberOfTimePeriodStd(order_number_series):
    if order_number_series.count() <= 1:
        return 0
    else:
        sort_series = order_number_series.sort_values().reset_index(drop=True)
        intervals = []
        for i in range(1, sort_series.count()):
            intervals.append(sort_series[i] - sort_series[i - 1])
        return np.std(intervals)

In [36]:
bought_number_of_time_period_mean_df = base_feature_df.groupby(['user_id', 'product_id'])['order_number'].agg(calculateBoughtNumberOfTimePeriodMean)

In [37]:
bought_number_of_time_period_mean_df

user_id  product_id
1        196             1.000000
         10258           1.000000
         10326         100.000000
         12427           1.000000
         13032           4.000000
         13176           3.000000
         14084         100.000000
         17122         100.000000
         25133           1.000000
         26088           1.000000
         26405           3.000000
         30450         100.000000
         35951         100.000000
         38928         100.000000
         39657         100.000000
         41787         100.000000
         46149           1.000000
         49235           1.000000
2        23            100.000000
         79            100.000000
         1559            1.800000
         2002            1.000000
         2573            4.000000
         3151          100.000000
         4071          100.000000
         4957          100.000000
         5212          100.000000
         5322          100.000000
         5450          100.0

In [ ]:
base_order_feature_df = pd.merge(order_df, order_products_prior_df, how='left', on='')